# 20% paper

In [1]:
import numpy as np

In [8]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_score, recall_score,
                             confusion_matrix,roc_auc_score,
                             roc_curve,fbeta_score
)
class FireflyOptimizedMLP:
    def __init__(self, input_dim, hidden_dims, output_dim, n_fireflies=20, 
                 max_iterations=100, alpha=0.2, beta0=1.0, gamma=1.0):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.n_fireflies = n_fireflies
        self.max_iterations = max_iterations
        self.alpha = alpha  # randomization parameter
        self.beta0 = beta0  # attractiveness at distance = 0
        self.gamma = gamma  # light absorption coefficient
        
        # Initialize model
        self.model = self._create_model()
        
        # Initialize firefly population
        self.fireflies = self._initialize_fireflies()
        self.best_firefly = None
        self.best_fitness = float('-inf')
        
    def _create_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=(self.input_dim,)))
        
        for dim in self.hidden_dims:
            model.add(tf.keras.layers.Dense(dim, activation='relu'))
            
        model.add(tf.keras.layers.Dense(self.output_dim, activation='softmax'))
        
        return model
    
    def _initialize_fireflies(self):
        fireflies = []
        weights_shape = self._get_weights_shape()
        
        for _ in range(self.n_fireflies):
            weights = []
            for shape in weights_shape:
                weights.append(np.random.uniform(-1, 1, shape))
            fireflies.append(weights)
            
        return fireflies
    
    def _get_weights_shape(self):
        return [w.shape for w in self.model.get_weights()]
    
    def _set_weights(self, weights):
        self.model.set_weights(weights)
    
    def _calculate_fitness(self, X, y):
        predictions = self.model.predict(X, verbose=0)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        return accuracy
    
    def _calculate_distance(self, firefly1, firefly2):
        total_distance = 0
        for w1, w2 in zip(firefly1, firefly2):
            total_distance += np.sum((w1 - w2) ** 2)
        return np.sqrt(total_distance)
    
    def _move_firefly(self, firefly1, firefly2):
        new_weights = []
        distance = self._calculate_distance(firefly1, firefly2)
        beta = self.beta0 * np.exp(-self.gamma * distance ** 2)
        
        for w1, w2 in zip(firefly1, firefly2):
            # Movement towards brighter firefly
            movement = beta * (w2 - w1)
            # Random movement
            random_movement = self.alpha * (np.random.uniform(-1, 1, w1.shape))
            new_weight = w1 + movement + random_movement
            new_weights.append(new_weight)
            
        return new_weights
    
    def fit(self, X, y, X_val=None, y_val=None):
        if X_val is None or y_val is None:
            X, X_val, y, y_val = train_test_split(X, y, test_size=0.2)
            
        history = {'train_accuracy': [], 'val_accuracy': []}
        
        for iteration in range(self.max_iterations):
            # Evaluate all fireflies
            fitness_values = []
            for firefly in self.fireflies:
                self._set_weights(firefly)
                fitness = self._calculate_fitness(X, y)
                fitness_values.append(fitness)
                
                if fitness > self.best_fitness:
                    self.best_fitness = fitness
                    self.best_firefly = firefly.copy()
            
            # Move fireflies
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if fitness_values[j] > fitness_values[i]:
                        self.fireflies[i] = self._move_firefly(
                            self.fireflies[i], 
                            self.fireflies[j]
                        )
            
            # Set best weights and evaluate
            self._set_weights(self.best_firefly)
            train_accuracy = self._calculate_fitness(X, y)
            val_accuracy = self._calculate_fitness(X_val, y_val)
            
            history['train_accuracy'].append(train_accuracy)
            history['val_accuracy'].append(val_accuracy)
            
            if (iteration + 1) % 10 == 0:
                print(f"Iteration {iteration + 1}/{self.max_iterations}")
                print(f"Train Accuracy: {train_accuracy:.4f}")
                print(f"Validation Accuracy: {val_accuracy:.4f}")
                
        return history
    
    def predict(self, X):
        self._set_weights(self.best_firefly)
        return self.model.predict(X)

# Example usage
def prepare_data():
    data = pd.read_csv("cm1.csv")
    data['defects'] = data['defects'].apply(lambda x:1 if x==True else 0)
    X = data.drop(['defects'],axis=1)
    y = data['defects']
    y = tf.keras.utils.to_categorical(y)
    
    # scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return train_test_split(X, y, test_size=0.3, random_state=21)

def main():
    #prepare data
    X_train, X_test, y_train, y_test = prepare_data()
    
    model = FireflyOptimizedMLP(
        input_dim=X_train.shape[1],
        hidden_dims=[64, 32],
        output_dim=y_train.shape[1],
        n_fireflies=50,
        max_iterations=50
    )
    
    # Train the model
    history = model.fit(X_train, y_train, X_test, y_test)
    
    # Make predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    test_accuracy = np.mean(predicted_classes == true_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    
    print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")
    print(f"Final Precision: {precision:.4f}")
    print(f"Final Recall: {recall:.4f}")
    
if __name__ == "__main__":
    main()

Iteration 10/50
Train Accuracy: 0.8937
Validation Accuracy: 0.9200
Iteration 20/50
Train Accuracy: 0.8937
Validation Accuracy: 0.9200
Iteration 30/50
Train Accuracy: 0.8937
Validation Accuracy: 0.9200
Iteration 40/50
Train Accuracy: 0.8937
Validation Accuracy: 0.9200
Iteration 50/50
Train Accuracy: 0.8937
Validation Accuracy: 0.9200
5/5 [==============================] - 0s 0s/step

Final Test Accuracy: 0.9200
Final Precision: 0.8583
Final Recall: 0.9200


In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

class FireflyOptimizedMLP:
    def __init__(self, input_dim, hidden_dims, output_dim, n_fireflies=20, 
                 max_iterations=100, alpha=0.2, beta0=1.0, gamma=1.0):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.n_fireflies = n_fireflies
        self.max_iterations = max_iterations
        self.alpha = alpha  # randomization parameter
        self.beta0 = beta0  # attractiveness at distance = 0
        self.gamma = gamma  # light absorption coefficient
        
        # Initialize model
        self.model = self._create_model()
        
        # Initialize firefly population
        self.fireflies = self._initialize_fireflies()
        self.best_firefly = None
        self.best_fitness = float('-inf')
        
    def _create_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=(self.input_dim,)))
        
        for dim in self.hidden_dims:
            model.add(tf.keras.layers.Dense(dim, activation='relu'))
            
        model.add(tf.keras.layers.Dense(self.output_dim, activation='softmax'))
        
        return model
    
    def _initialize_fireflies(self):
        fireflies = []
        weights_shape = self._get_weights_shape()
        
        for _ in range(self.n_fireflies):
            weights = []
            for shape in weights_shape:
                weights.append(np.random.uniform(-1, 1, shape))
            fireflies.append(weights)
            
        return fireflies
    
    def _get_weights_shape(self):
        return [w.shape for w in self.model.get_weights()]
    
    def _set_weights(self, weights):
        self.model.set_weights(weights)
    
    def _calculate_fitness(self, X, y):
        predictions = self.model.predict(X, verbose=0)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        return accuracy
    
    def _calculate_distance(self, firefly1, firefly2):
        total_distance = 0
        for w1, w2 in zip(firefly1, firefly2):
            total_distance += np.sum((w1 - w2) ** 2)
        return np.sqrt(total_distance)
    
    def _move_firefly(self, firefly1, firefly2):
        new_weights = []
        distance = self._calculate_distance(firefly1, firefly2)
        beta = self.beta0 * np.exp(-self.gamma * distance ** 2)
        
        for w1, w2 in zip(firefly1, firefly2):
            # Movement towards brighter firefly
            movement = beta * (w2 - w1)
            # Random movement
            random_movement = self.alpha * (np.random.uniform(-1, 1, w1.shape))
            new_weight = w1 + movement + random_movement
            new_weights.append(new_weight)
            
        return new_weights
    
    def fit(self, X, y, X_val=None, y_val=None):
        if X_val is None or y_val is None:
            X, X_val, y, y_val = train_test_split(X, y, test_size=0.2)
            
        history = {'train_accuracy': [], 'val_accuracy': []}
        
        for iteration in range(self.max_iterations):
            # Evaluate all fireflies
            fitness_values = []
            for firefly in self.fireflies:
                self._set_weights(firefly)
                fitness = self._calculate_fitness(X, y)
                fitness_values.append(fitness)
                
                if fitness > self.best_fitness:
                    self.best_fitness = fitness
                    self.best_firefly = firefly.copy()
            
            # Move fireflies
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if fitness_values[j] > fitness_values[i]:
                        self.fireflies[i] = self._move_firefly(
                            self.fireflies[i], 
                            self.fireflies[j]
                        )
            
            # Set best weights and evaluate
            self._set_weights(self.best_firefly)
            train_accuracy = self._calculate_fitness(X, y)
            val_accuracy = self._calculate_fitness(X_val, y_val)
            
            history['train_accuracy'].append(train_accuracy)
            history['val_accuracy'].append(val_accuracy)
            
            if (iteration + 1) % 10 == 0:
                print(f"Iteration {iteration + 1}/{self.max_iterations}")
                print(f"Train Accuracy: {train_accuracy:.4f}")
                print(f"Validation Accuracy: {val_accuracy:.4f}")
                
        return history
    
    def predict(self, X):
        self._set_weights(self.best_firefly)
        return self.model.predict(X)

# Example usage
def prepare_data():
    data = pd.read_csv("pc1.csv")
    data['defects'] = data['defects'].apply(lambda x:1 if x==True else 0)
    X = data.drop(['defects'],axis=1)
    y = data['defects']
    y = tf.keras.utils.to_categorical(y)
    
    # scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return train_test_split(X, y, test_size=0.3, random_state=21)

def main():
    #prepare data
    X_train, X_test, y_train, y_test = prepare_data()
    
    model = FireflyOptimizedMLP(
        input_dim=X_train.shape[1],
        hidden_dims=[64, 32],
        output_dim=y_train.shape[1],
        n_fireflies=50,
        max_iterations=50
    )
    
    # Train the model
    history = model.fit(X_train, y_train, X_test, y_test)
    
    # Make predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    test_accuracy = np.mean(predicted_classes == true_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    
    print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")
    print(f"Final Precision: {precision:.4f}")
    print(f"Final Recall: {recall:.4f}")
    
if __name__ == "__main__":
    main()

Iteration 10/50
Train Accuracy: 0.9278
Validation Accuracy: 0.9369
Iteration 20/50
Train Accuracy: 0.9278
Validation Accuracy: 0.9369
Iteration 30/50
Train Accuracy: 0.9278
Validation Accuracy: 0.9369
Iteration 40/50
Train Accuracy: 0.9278
Validation Accuracy: 0.9369
Iteration 50/50
Train Accuracy: 0.9278
Validation Accuracy: 0.9369
11/11 [==============================] - 0s 2ms/step

Final Test Accuracy: 0.9369
Final Precision: 0.8779
Final Recall: 0.9369


C:\Users\kanis\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

class FireflyOptimizedMLP:
    def __init__(self, input_dim, hidden_dims, output_dim, n_fireflies=20, 
                 max_iterations=100, alpha=0.2, beta0=1.0, gamma=1.0):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.n_fireflies = n_fireflies
        self.max_iterations = max_iterations
        self.alpha = alpha  # randomization parameter
        self.beta0 = beta0  # attractiveness at distance = 0
        self.gamma = gamma  # light absorption coefficient
        
        # Initialize model
        self.model = self._create_model()
        
        # Initialize firefly population
        self.fireflies = self._initialize_fireflies()
        self.best_firefly = None
        self.best_fitness = float('-inf')
        
    def _create_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=(self.input_dim,)))
        
        for dim in self.hidden_dims:
            model.add(tf.keras.layers.Dense(dim, activation='relu'))
            
        model.add(tf.keras.layers.Dense(self.output_dim, activation='softmax'))
        
        return model
    
    def _initialize_fireflies(self):
        fireflies = []
        weights_shape = self._get_weights_shape()
        
        for _ in range(self.n_fireflies):
            weights = []
            for shape in weights_shape:
                weights.append(np.random.uniform(-1, 1, shape))
            fireflies.append(weights)
            
        return fireflies
    
    def _get_weights_shape(self):
        return [w.shape for w in self.model.get_weights()]
    
    def _set_weights(self, weights):
        self.model.set_weights(weights)
    
    def _calculate_fitness(self, X, y):
        predictions = self.model.predict(X, verbose=0)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        return accuracy
    
    def _calculate_distance(self, firefly1, firefly2):
        total_distance = 0
        for w1, w2 in zip(firefly1, firefly2):
            total_distance += np.sum((w1 - w2) ** 2)
        return np.sqrt(total_distance)
    
    def _move_firefly(self, firefly1, firefly2):
        new_weights = []
        distance = self._calculate_distance(firefly1, firefly2)
        beta = self.beta0 * np.exp(-self.gamma * distance ** 2)
        
        for w1, w2 in zip(firefly1, firefly2):
            # Movement towards brighter firefly
            movement = beta * (w2 - w1)
            # Random movement
            random_movement = self.alpha * (np.random.uniform(-1, 1, w1.shape))
            new_weight = w1 + movement + random_movement
            new_weights.append(new_weight)
            
        return new_weights
    
    def fit(self, X, y, X_val=None, y_val=None):
        if X_val is None or y_val is None:
            X, X_val, y, y_val = train_test_split(X, y, test_size=0.2)
            
        history = {'train_accuracy': [], 'val_accuracy': []}
        
        for iteration in range(self.max_iterations):
            # Evaluate all fireflies
            fitness_values = []
            for firefly in self.fireflies:
                self._set_weights(firefly)
                fitness = self._calculate_fitness(X, y)
                fitness_values.append(fitness)
                
                if fitness > self.best_fitness:
                    self.best_fitness = fitness
                    self.best_firefly = firefly.copy()
            
            # Move fireflies
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if fitness_values[j] > fitness_values[i]:
                        self.fireflies[i] = self._move_firefly(
                            self.fireflies[i], 
                            self.fireflies[j]
                        )
            
            # Set best weights and evaluate
            self._set_weights(self.best_firefly)
            train_accuracy = self._calculate_fitness(X, y)
            val_accuracy = self._calculate_fitness(X_val, y_val)
            
            history['train_accuracy'].append(train_accuracy)
            history['val_accuracy'].append(val_accuracy)
            
            if (iteration + 1) % 10 == 0:
                print(f"Iteration {iteration + 1}/{self.max_iterations}")
                print(f"Train Accuracy: {train_accuracy:.4f}")
                print(f"Validation Accuracy: {val_accuracy:.4f}")
                
        return history
    
    def predict(self, X):
        self._set_weights(self.best_firefly)
        return self.model.predict(X)

# Example usage
def prepare_data():
    data = pd.read_csv("kc1.csv")
    data['defects'] = data['defects'].apply(lambda x:1 if x==True else 0)
    X = data.drop(['defects'],axis=1)
    y = data['defects']
    y = tf.keras.utils.to_categorical(y)
    
    # scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return train_test_split(X, y, test_size=0.3, random_state=21)

def main():
    #prepare data
    X_train, X_test, y_train, y_test = prepare_data()
    
    model = FireflyOptimizedMLP(
        input_dim=X_train.shape[1],
        hidden_dims=[64, 32],
        output_dim=y_train.shape[1],
        n_fireflies=60,
        max_iterations=50
    )
    
    # Train the model
    history = model.fit(X_train, y_train, X_test, y_test)
    
    # Make predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    test_accuracy = np.mean(predicted_classes == true_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    
    print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")
    print(f"Final Precision: {precision:.4f}")
    print(f"Final Recall: {recall:.4f}")
    
if __name__ == "__main__":
    main()

Iteration 10/50
Train Accuracy: 0.8496
Validation Accuracy: 0.8531
Iteration 20/50
Train Accuracy: 0.8496
Validation Accuracy: 0.8531
Iteration 30/50
Train Accuracy: 0.8503
Validation Accuracy: 0.8610
Iteration 40/50
Train Accuracy: 0.8543
Validation Accuracy: 0.8610
Iteration 50/50
Train Accuracy: 0.8543
Validation Accuracy: 0.8610
20/20 [==============================] - 0s 2ms/step

Final Test Accuracy: 0.8610
Final Precision: 0.8344
Final Recall: 0.8610


In [7]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

class FireflyOptimizedMLP:
    def __init__(self, input_dim, hidden_dims, output_dim, n_fireflies=20, 
                 max_iterations=100, alpha=0.2, beta0=1.0, gamma=1.0):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.n_fireflies = n_fireflies
        self.max_iterations = max_iterations
        self.alpha = alpha  # randomization parameter
        self.beta0 = beta0  # attractiveness at distance = 0
        self.gamma = gamma  # light absorption coefficient
        
        # Initialize model
        self.model = self._create_model()
        
        # Initialize firefly population
        self.fireflies = self._initialize_fireflies()
        self.best_firefly = None
        self.best_fitness = float('-inf')
        
    def _create_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=(self.input_dim,)))
        
        for dim in self.hidden_dims:
            model.add(tf.keras.layers.Dense(dim, activation='relu'))
            
        model.add(tf.keras.layers.Dense(self.output_dim, activation='softmax'))
        
        return model
    
    def _initialize_fireflies(self):
        fireflies = []
        weights_shape = self._get_weights_shape()
        
        for _ in range(self.n_fireflies):
            weights = []
            for shape in weights_shape:
                weights.append(np.random.uniform(-1, 1, shape))
            fireflies.append(weights)
            
        return fireflies
    
    def _get_weights_shape(self):
        return [w.shape for w in self.model.get_weights()]
    
    def _set_weights(self, weights):
        self.model.set_weights(weights)
    
    def _calculate_fitness(self, X, y):
        predictions = self.model.predict(X, verbose=0)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        return accuracy
    
    def _calculate_distance(self, firefly1, firefly2):
        total_distance = 0
        for w1, w2 in zip(firefly1, firefly2):
            total_distance += np.sum((w1 - w2) ** 2)
        return np.sqrt(total_distance)
    
    def _move_firefly(self, firefly1, firefly2):
        new_weights = []
        distance = self._calculate_distance(firefly1, firefly2)
        beta = self.beta0 * np.exp(-self.gamma * distance ** 2)
        
        for w1, w2 in zip(firefly1, firefly2):
            # Movement towards brighter firefly
            movement = beta * (w2 - w1)
            # Random movement
            random_movement = self.alpha * (np.random.uniform(-1, 1, w1.shape))
            new_weight = w1 + movement + random_movement
            new_weights.append(new_weight)
            
        return new_weights
    
    def fit(self, X, y, X_val=None, y_val=None):
        if X_val is None or y_val is None:
            X, X_val, y, y_val = train_test_split(X, y, test_size=0.2)
            
        history = {'train_accuracy': [], 'val_accuracy': []}
        
        for iteration in range(self.max_iterations):
            # Evaluate all fireflies
            fitness_values = []
            for firefly in self.fireflies:
                self._set_weights(firefly)
                fitness = self._calculate_fitness(X, y)
                fitness_values.append(fitness)
                
                if fitness > self.best_fitness:
                    self.best_fitness = fitness
                    self.best_firefly = firefly.copy()
            
            # Move fireflies
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if fitness_values[j] > fitness_values[i]:
                        self.fireflies[i] = self._move_firefly(
                            self.fireflies[i], 
                            self.fireflies[j]
                        )
            
            # Set best weights and evaluate
            self._set_weights(self.best_firefly)
            train_accuracy = self._calculate_fitness(X, y)
            val_accuracy = self._calculate_fitness(X_val, y_val)
            
            history['train_accuracy'].append(train_accuracy)
            history['val_accuracy'].append(val_accuracy)
            
            if (iteration + 1) % 10 == 0:
                print(f"Iteration {iteration + 1}/{self.max_iterations}")
                print(f"Train Accuracy: {train_accuracy:.4f}")
                print(f"Validation Accuracy: {val_accuracy:.4f}")
                
        return history
    
    def predict(self, X):
        self._set_weights(self.best_firefly)
        return self.model.predict(X)

# Example usage
def prepare_data():
    data = pd.read_csv("kc2.csv")
    data['defects'] = data['problems'].apply(lambda x:1 if x==True else 0)
    X = data.drop(['problems'],axis=1)
    y = data['defects']
    y = tf.keras.utils.to_categorical(y)
    
    # scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return train_test_split(X, y, test_size=0.3, random_state=21)

def main():
    #prepare data
    X_train, X_test, y_train, y_test = prepare_data()
    
    model = FireflyOptimizedMLP(
        input_dim=X_train.shape[1],
        hidden_dims=[64, 32],
        output_dim=y_train.shape[1],
        n_fireflies=50,
        max_iterations=50
    )
    
    # Train the model
    history = model.fit(X_train, y_train, X_test, y_test)
    
    # Make predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    test_accuracy = np.mean(predicted_classes == true_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    
    print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")
    print(f"Final Precision: {precision:.4f}")
    print(f"Final Recall: {recall:.4f}")
    
if __name__ == "__main__":
    main()

Iteration 10/50
Train Accuracy: 1.0000
Validation Accuracy: 1.0000
Iteration 20/50
Train Accuracy: 1.0000
Validation Accuracy: 1.0000
Iteration 30/50
Train Accuracy: 1.0000
Validation Accuracy: 1.0000
Iteration 40/50
Train Accuracy: 1.0000
Validation Accuracy: 1.0000
Iteration 50/50
Train Accuracy: 1.0000
Validation Accuracy: 1.0000
5/5 [==============================] - 0s 0s/step

Final Test Accuracy: 1.0000
Final Precision: 1.0000
Final Recall: 1.0000


# 50% paper

In [8]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_score, recall_score,
                             confusion_matrix, roc_auc_score,
                             roc_curve, fbeta_score
)

class FireflyOptimizedMLP:
    def __init__(self, input_dim, hidden_dims, output_dim, n_fireflies=20, 
                 max_iterations=100, alpha=0.2, beta0=1.0, gamma=1.0):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.n_fireflies = n_fireflies
        self.max_iterations = max_iterations
        self.alpha = alpha  # randomization parameter
        self.beta0 = beta0  # attractiveness at distance = 0
        self.gamma = gamma  # light absorption coefficient
        
        # Initialize model
        self.model = self._create_model()
        
        # Initialize firefly population
        self.fireflies = self._initialize_fireflies()
        self.best_firefly = None
        self.best_fitness = float('-inf')
        
    def _create_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=(self.input_dim,)))
        
        for dim in self.hidden_dims:
            model.add(tf.keras.layers.Dense(dim, activation='relu'))
            
        model.add(tf.keras.layers.Dense(self.output_dim, activation='softmax'))
        
        return model
    
    def _initialize_fireflies(self):
        fireflies = []
        weights_shape = self._get_weights_shape()
        
        for _ in range(self.n_fireflies):
            weights = []
            for shape in weights_shape:
                weights.append(np.random.uniform(-1, 1, shape))
            fireflies.append(weights)
            
        return fireflies
    
    def _get_weights_shape(self):
        return [w.shape for w in self.model.get_weights()]
    
    def _set_weights(self, weights):
        self.model.set_weights(weights)
    
    def _calculate_fitness(self, X, y):
        predictions = self.model.predict(X, verbose=0)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        return accuracy
    
    def _calculate_distance(self, firefly1, firefly2):
        total_distance = 0
        for w1, w2 in zip(firefly1, firefly2):
            total_distance += np.sum((w1 - w2) ** 2)
        return np.sqrt(total_distance)
    
    def _move_firefly(self, firefly1, firefly2):
        new_weights = []
        distance = self._calculate_distance(firefly1, firefly2)
        beta = self.beta0 * np.exp(-self.gamma * distance ** 2)
        
        for w1, w2 in zip(firefly1, firefly2):
            # Movement towards brighter firefly
            movement = beta * (w2 - w1)
            # Random movement
            random_movement = self.alpha * (np.random.uniform(-1, 1, w1.shape))
            new_weight = w1 + movement + random_movement
            new_weights.append(new_weight)
            
        return new_weights
    
    def fit(self, X, y, X_val=None, y_val=None):
        if X_val is None or y_val is None:
            X, X_val, y, y_val = train_test_split(X, y, test_size=0.2)
            
        history = {'train_accuracy': [], 'val_accuracy': []}
        
        for iteration in range(self.max_iterations):
            # Evaluate all fireflies
            fitness_values = []
            for firefly in self.fireflies:
                self._set_weights(firefly)
                fitness = self._calculate_fitness(X, y)
                fitness_values.append(fitness)
                
                if fitness > self.best_fitness:
                    self.best_fitness = fitness
                    self.best_firefly = firefly.copy()
            
            # Move fireflies
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if fitness_values[j] > fitness_values[i]:
                        self.fireflies[i] = self._move_firefly(
                            self.fireflies[i], 
                            self.fireflies[j]
                        )
            
            # Set best weights and evaluate
            self._set_weights(self.best_firefly)
            train_accuracy = self._calculate_fitness(X, y)
            val_accuracy = self._calculate_fitness(X_val, y_val)
            
            history['train_accuracy'].append(train_accuracy)
            history['val_accuracy'].append(val_accuracy)
            
            if (iteration + 1) % 10 == 0:
                print(f"Iteration {iteration + 1}/{self.max_iterations}")
                print(f"Train Accuracy: {train_accuracy:.4f}")
                print(f"Validation Accuracy: {val_accuracy:.4f}")
                
        return history
    
    def predict(self, X):
        self._set_weights(self.best_firefly)
        return self.model.predict(X)

# Example usage
def prepare_data():
    data = pd.read_csv("cm1.csv")
    data['defects'] = data['defects'].apply(lambda x:1 if x==True else 0)
    X = data.drop(['defects'],axis=1)
    y = data['defects']
    y = tf.keras.utils.to_categorical(y)
    
    # scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return train_test_split(X, y, test_size=0.3, random_state=21)

def main():
    #prepare data
    X_train, X_test, y_train, y_test = prepare_data()
    
    model = FireflyOptimizedMLP(
        input_dim=X_train.shape[1],
        hidden_dims=[64, 32],
        output_dim=y_train.shape[1],
        n_fireflies=50,
        max_iterations=50
    )
    
    # Train the model
    history = model.fit(X_train, y_train, X_test, y_test)
    
    # Make predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    test_accuracy = np.mean(predicted_classes == true_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    
    # Confusion Matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    
    # ROC AUC Score (using probabilistic predictions)
    roc_auc = roc_auc_score(y_test, predictions)
    
    # ROC Curve (returned but not plotted in this example)
    fpr, tpr, thresholds = roc_curve(y_test.ravel(), predictions.ravel())
    
    # F-beta Score (with beta=1, which is equivalent to F1 score)
    fbeta = fbeta_score(true_classes, predicted_classes, beta=1, average='weighted')
    
    print(f"\nFinal Test Metrics:")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"F-beta Score (F1): {fbeta:.4f}")
    print(f"\nROC Curve points available ({fpr}, {tpr}, {thresholds})")
    
if __name__ == "__main__":
    main()

Iteration 10/50
Train Accuracy: 0.8937
Validation Accuracy: 0.9267
Iteration 20/50
Train Accuracy: 0.8937
Validation Accuracy: 0.9267
Iteration 30/50
Train Accuracy: 0.8937
Validation Accuracy: 0.9267
Iteration 40/50
Train Accuracy: 0.8937
Validation Accuracy: 0.9267
Iteration 50/50
Train Accuracy: 0.8937
Validation Accuracy: 0.9267
5/5 [==============================] - 0s 4ms/step

Final Test Metrics:
Test Accuracy: 0.9267
Precision: 0.8587
Recall: 0.9267
Confusion Matrix:
[[139   0]
 [ 11   0]]
ROC AUC Score: 0.4591
F-beta Score (F1): 0.8914

ROC Curve points available ([0.         0.07333333 0.09333333 0.09333333 0.25333333 0.25333333
 0.34666667 0.36       0.39333333 0.40666667 0.44       0.44
 0.51333333 0.51333333 0.52       0.53333333 0.55333333 1.        ], [0.         0.92666667 0.92666667 0.93333333 0.93333333 0.94
 0.94       0.94       0.94       0.94       0.94       0.94666667
 0.94666667 0.95333333 0.95333333 0.95333333 0.95333333 1.        ], [2.0000000e+00 1.0000000e+

C:\Users\kanis\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_score, recall_score,
                             confusion_matrix, roc_auc_score,
                             roc_curve, fbeta_score
)

class FireflyOptimizedMLP:
    def __init__(self, input_dim, hidden_dims, output_dim, n_fireflies=20, 
                 max_iterations=100, alpha=0.2, beta0=1.0, gamma=1.0):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.n_fireflies = n_fireflies
        self.max_iterations = max_iterations
        self.alpha = alpha  # randomization parameter
        self.beta0 = beta0  # attractiveness at distance = 0
        self.gamma = gamma  # light absorption coefficient
        
        # Initialize model
        self.model = self._create_model()
        
        # Initialize firefly population
        self.fireflies = self._initialize_fireflies()
        self.best_firefly = None
        self.best_fitness = float('-inf')
        
    def _create_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=(self.input_dim,)))
        
        for dim in self.hidden_dims:
            model.add(tf.keras.layers.Dense(dim, activation='relu'))
            
        model.add(tf.keras.layers.Dense(self.output_dim, activation='softmax'))
        
        return model
    
    def _initialize_fireflies(self):
        fireflies = []
        weights_shape = self._get_weights_shape()
        
        for _ in range(self.n_fireflies):
            weights = []
            for shape in weights_shape:
                weights.append(np.random.uniform(-1, 1, shape))
            fireflies.append(weights)
            
        return fireflies
    
    def _get_weights_shape(self):
        return [w.shape for w in self.model.get_weights()]
    
    def _set_weights(self, weights):
        self.model.set_weights(weights)
    
    def _calculate_fitness(self, X, y):
        predictions = self.model.predict(X, verbose=0)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        return accuracy
    
    def _calculate_distance(self, firefly1, firefly2):
        total_distance = 0
        for w1, w2 in zip(firefly1, firefly2):
            total_distance += np.sum((w1 - w2) ** 2)
        return np.sqrt(total_distance)
    
    def _move_firefly(self, firefly1, firefly2):
        new_weights = []
        distance = self._calculate_distance(firefly1, firefly2)
        beta = self.beta0 * np.exp(-self.gamma * distance ** 2)
        
        for w1, w2 in zip(firefly1, firefly2):
            # Movement towards brighter firefly
            movement = beta * (w2 - w1)
            # Random movement
            random_movement = self.alpha * (np.random.uniform(-1, 1, w1.shape))
            new_weight = w1 + movement + random_movement
            new_weights.append(new_weight)
            
        return new_weights
    
    def fit(self, X, y, X_val=None, y_val=None):
        if X_val is None or y_val is None:
            X, X_val, y, y_val = train_test_split(X, y, test_size=0.2)
            
        history = {'train_accuracy': [], 'val_accuracy': []}
        
        for iteration in range(self.max_iterations):
            # Evaluate all fireflies
            fitness_values = []
            for firefly in self.fireflies:
                self._set_weights(firefly)
                fitness = self._calculate_fitness(X, y)
                fitness_values.append(fitness)
                
                if fitness > self.best_fitness:
                    self.best_fitness = fitness
                    self.best_firefly = firefly.copy()
            
            # Move fireflies
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if fitness_values[j] > fitness_values[i]:
                        self.fireflies[i] = self._move_firefly(
                            self.fireflies[i], 
                            self.fireflies[j]
                        )
            
            # Set best weights and evaluate
            self._set_weights(self.best_firefly)
            train_accuracy = self._calculate_fitness(X, y)
            val_accuracy = self._calculate_fitness(X_val, y_val)
            
            history['train_accuracy'].append(train_accuracy)
            history['val_accuracy'].append(val_accuracy)
            
            if (iteration + 1) % 10 == 0:
                print(f"Iteration {iteration + 1}/{self.max_iterations}")
                print(f"Train Accuracy: {train_accuracy:.4f}")
                print(f"Validation Accuracy: {val_accuracy:.4f}")
                
        return history
    
    def predict(self, X):
        self._set_weights(self.best_firefly)
        return self.model.predict(X)

# Example usage
def prepare_data():
    data = pd.read_csv("pc1.csv")
    data['defects'] = data['defects'].apply(lambda x:1 if x==True else 0)
    X = data.drop(['defects'],axis=1)
    y = data['defects']
    y = tf.keras.utils.to_categorical(y)
    
    # scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return train_test_split(X, y, test_size=0.3, random_state=21)

def main():
    #prepare data
    X_train, X_test, y_train, y_test = prepare_data()
    
    model = FireflyOptimizedMLP(
        input_dim=X_train.shape[1],
        hidden_dims=[64, 32],
        output_dim=y_train.shape[1],
        n_fireflies=50,
        max_iterations=50
    )
    
    # Train the model
    history = model.fit(X_train, y_train, X_test, y_test)
    
    # Make predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    test_accuracy = np.mean(predicted_classes == true_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    
    # Confusion Matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    
    # ROC AUC Score (using probabilistic predictions)
    roc_auc = roc_auc_score(y_test, predictions)
    
    # ROC Curve (returned but not plotted in this example)
    fpr, tpr, thresholds = roc_curve(y_test.ravel(), predictions.ravel())
    
    # F-beta Score (with beta=1, which is equivalent to F1 score)
    fbeta = fbeta_score(true_classes, predicted_classes, beta=1, average='weighted')
    
    print(f"\nFinal Test Metrics:")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"F-beta Score (F1): {fbeta:.4f}")
    print(f"\nROC Curve points available ({fpr}, {tpr}, {thresholds})")
    
if __name__ == "__main__":
    main()

Iteration 10/50
Train Accuracy: 0.9291
Validation Accuracy: 0.9369
Iteration 20/50
Train Accuracy: 0.9291
Validation Accuracy: 0.9369
Iteration 30/50
Train Accuracy: 0.9291
Validation Accuracy: 0.9369
Iteration 40/50
Train Accuracy: 0.9291
Validation Accuracy: 0.9369
Iteration 50/50
Train Accuracy: 0.9291
Validation Accuracy: 0.9369
11/11 [==============================] - 0s 3ms/step

Final Test Metrics:
Test Accuracy: 0.9369
Precision: 0.8779
Recall: 0.9369
Confusion Matrix:
[[312   0]
 [ 21   0]]
ROC AUC Score: 0.5265
F-beta Score (F1): 0.9064

ROC Curve points available ([0.         0.06006006 0.06006006 0.06306306 0.06306306 0.06306306
 0.06306306 0.07207207 0.07807808 0.08408408 0.08408408 0.0960961
 0.0960961  0.11111111 0.11111111 0.17417417 0.17417417 0.18018018
 0.18018018 0.23423423 0.23423423 0.26126126 0.27027027 0.3033033
 0.3033033  0.33333333 0.33933934 0.34834835 0.34834835 0.36636637
 0.37237237 0.37537538 0.37537538 0.38138138 0.39339339 1.        ], [0.         0.90

C:\Users\kanis\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_score, recall_score,
                             confusion_matrix, roc_auc_score,
                             roc_curve, fbeta_score
)

class FireflyOptimizedMLP:
    def __init__(self, input_dim, hidden_dims, output_dim, n_fireflies=20, 
                 max_iterations=100, alpha=0.2, beta0=1.0, gamma=1.0):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.n_fireflies = n_fireflies
        self.max_iterations = max_iterations
        self.alpha = alpha  # randomization parameter
        self.beta0 = beta0  # attractiveness at distance = 0
        self.gamma = gamma  # light absorption coefficient
        
        # Initialize model
        self.model = self._create_model()
        
        # Initialize firefly population
        self.fireflies = self._initialize_fireflies()
        self.best_firefly = None
        self.best_fitness = float('-inf')
        
    def _create_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=(self.input_dim,)))
        
        for dim in self.hidden_dims:
            model.add(tf.keras.layers.Dense(dim, activation='relu'))
            
        model.add(tf.keras.layers.Dense(self.output_dim, activation='softmax'))
        
        return model
    
    def _initialize_fireflies(self):
        fireflies = []
        weights_shape = self._get_weights_shape()
        
        for _ in range(self.n_fireflies):
            weights = []
            for shape in weights_shape:
                weights.append(np.random.uniform(-1, 1, shape))
            fireflies.append(weights)
            
        return fireflies
    
    def _get_weights_shape(self):
        return [w.shape for w in self.model.get_weights()]
    
    def _set_weights(self, weights):
        self.model.set_weights(weights)
    
    def _calculate_fitness(self, X, y):
        predictions = self.model.predict(X, verbose=0)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        return accuracy
    
    def _calculate_distance(self, firefly1, firefly2):
        total_distance = 0
        for w1, w2 in zip(firefly1, firefly2):
            total_distance += np.sum((w1 - w2) ** 2)
        return np.sqrt(total_distance)
    
    def _move_firefly(self, firefly1, firefly2):
        new_weights = []
        distance = self._calculate_distance(firefly1, firefly2)
        beta = self.beta0 * np.exp(-self.gamma * distance ** 2)
        
        for w1, w2 in zip(firefly1, firefly2):
            # Movement towards brighter firefly
            movement = beta * (w2 - w1)
            # Random movement
            random_movement = self.alpha * (np.random.uniform(-1, 1, w1.shape))
            new_weight = w1 + movement + random_movement
            new_weights.append(new_weight)
            
        return new_weights
    
    def fit(self, X, y, X_val=None, y_val=None):
        if X_val is None or y_val is None:
            X, X_val, y, y_val = train_test_split(X, y, test_size=0.2)
            
        history = {'train_accuracy': [], 'val_accuracy': []}
        
        for iteration in range(self.max_iterations):
            # Evaluate all fireflies
            fitness_values = []
            for firefly in self.fireflies:
                self._set_weights(firefly)
                fitness = self._calculate_fitness(X, y)
                fitness_values.append(fitness)
                
                if fitness > self.best_fitness:
                    self.best_fitness = fitness
                    self.best_firefly = firefly.copy()
            
            # Move fireflies
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if fitness_values[j] > fitness_values[i]:
                        self.fireflies[i] = self._move_firefly(
                            self.fireflies[i], 
                            self.fireflies[j]
                        )
            
            # Set best weights and evaluate
            self._set_weights(self.best_firefly)
            train_accuracy = self._calculate_fitness(X, y)
            val_accuracy = self._calculate_fitness(X_val, y_val)
            
            history['train_accuracy'].append(train_accuracy)
            history['val_accuracy'].append(val_accuracy)
            
            if (iteration + 1) % 10 == 0:
                print(f"Iteration {iteration + 1}/{self.max_iterations}")
                print(f"Train Accuracy: {train_accuracy:.4f}")
                print(f"Validation Accuracy: {val_accuracy:.4f}")
                
        return history
    
    def predict(self, X):
        self._set_weights(self.best_firefly)
        return self.model.predict(X)

# Example usage
def prepare_data():
    data = pd.read_csv("kc1.csv")
    data['defects'] = data['defects'].apply(lambda x:1 if x==True else 0)
    X = data.drop(['defects'],axis=1)
    y = data['defects']
    y = tf.keras.utils.to_categorical(y)
    
    # scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return train_test_split(X, y, test_size=0.3, random_state=21)

def main():
    #prepare data
    X_train, X_test, y_train, y_test = prepare_data()
    
    model = FireflyOptimizedMLP(
        input_dim=X_train.shape[1],
        hidden_dims=[64, 32],
        output_dim=y_train.shape[1],
        n_fireflies=60,
        max_iterations=50
    )
    
    # Train the model
    history = model.fit(X_train, y_train, X_test, y_test)
    
    # Make predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    test_accuracy = np.mean(predicted_classes == true_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    
    # Confusion Matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    
    # ROC AUC Score (using probabilistic predictions)
    roc_auc = roc_auc_score(y_test, predictions)
    
    # ROC Curve (returned but not plotted in this example)
    fpr, tpr, thresholds = roc_curve(y_test.ravel(), predictions.ravel())
    
    # F-beta Score (with beta=1, which is equivalent to F1 score)
    fbeta = fbeta_score(true_classes, predicted_classes, beta=1, average='weighted')
    
    print(f"\nFinal Test Metrics:")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"F-beta Score (F1): {fbeta:.4f}")
    print(f"\nROC Curve points available ({fpr}, {tpr}, {thresholds})")
    
if __name__ == "__main__":
    main()

Iteration 10/50
Train Accuracy: 0.8482
Validation Accuracy: 0.8657
Iteration 20/50
Train Accuracy: 0.8482
Validation Accuracy: 0.8657
Iteration 30/50
Train Accuracy: 0.8482
Validation Accuracy: 0.8657
Iteration 40/50
Train Accuracy: 0.8523
Validation Accuracy: 0.8736
Iteration 50/50
Train Accuracy: 0.8543
Validation Accuracy: 0.8752
20/20 [==============================] - 0s 2ms/step

Final Test Metrics:
Test Accuracy: 0.8752
Precision: 0.8602
Recall: 0.8752
Confusion Matrix:
[[530   9]
 [ 70  24]]
ROC AUC Score: 0.6177
F-beta Score (F1): 0.8486

ROC Curve points available ([0.         0.12480253 0.12480253 0.12638231 0.12638231 0.12796209
 0.12796209 0.1342812  1.        ], [0.         0.87361769 0.87519747 0.87519747 0.87677725 0.87677725
 0.87835703 0.87835703 1.        ], [2.0000000e+00 1.0000000e+00 9.9999297e-01 7.0529290e-06 3.0003638e-12
 1.2762712e-14 9.6811807e-22 1.1952789e-29 0.0000000e+00])


In [12]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_score, recall_score,
                             confusion_matrix, roc_auc_score,
                             roc_curve, fbeta_score
)

class FireflyOptimizedMLP:
    def __init__(self, input_dim, hidden_dims, output_dim, n_fireflies=20, 
                 max_iterations=100, alpha=0.2, beta0=1.0, gamma=1.0):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.n_fireflies = n_fireflies
        self.max_iterations = max_iterations
        self.alpha = alpha  # randomization parameter
        self.beta0 = beta0  # attractiveness at distance = 0
        self.gamma = gamma  # light absorption coefficient
        
        # Initialize model
        self.model = self._create_model()
        
        # Initialize firefly population
        self.fireflies = self._initialize_fireflies()
        self.best_firefly = None
        self.best_fitness = float('-inf')
        
    def _create_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=(self.input_dim,)))
        
        for dim in self.hidden_dims:
            model.add(tf.keras.layers.Dense(dim, activation='relu'))
            
        model.add(tf.keras.layers.Dense(self.output_dim, activation='softmax'))
        
        return model
    
    def _initialize_fireflies(self):
        fireflies = []
        weights_shape = self._get_weights_shape()
        
        for _ in range(self.n_fireflies):
            weights = []
            for shape in weights_shape:
                weights.append(np.random.uniform(-1, 1, shape))
            fireflies.append(weights)
            
        return fireflies
    
    def _get_weights_shape(self):
        return [w.shape for w in self.model.get_weights()]
    
    def _set_weights(self, weights):
        self.model.set_weights(weights)
    
    def _calculate_fitness(self, X, y):
        predictions = self.model.predict(X, verbose=0)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        return accuracy
    
    def _calculate_distance(self, firefly1, firefly2):
        total_distance = 0
        for w1, w2 in zip(firefly1, firefly2):
            total_distance += np.sum((w1 - w2) ** 2)
        return np.sqrt(total_distance)
    
    def _move_firefly(self, firefly1, firefly2):
        new_weights = []
        distance = self._calculate_distance(firefly1, firefly2)
        beta = self.beta0 * np.exp(-self.gamma * distance ** 2)
        
        for w1, w2 in zip(firefly1, firefly2):
            # Movement towards brighter firefly
            movement = beta * (w2 - w1)
            # Random movement
            random_movement = self.alpha * (np.random.uniform(-1, 1, w1.shape))
            new_weight = w1 + movement + random_movement
            new_weights.append(new_weight)
            
        return new_weights
    
    def fit(self, X, y, X_val=None, y_val=None):
        if X_val is None or y_val is None:
            X, X_val, y, y_val = train_test_split(X, y, test_size=0.2)
            
        history = {'train_accuracy': [], 'val_accuracy': []}
        
        for iteration in range(self.max_iterations):
            # Evaluate all fireflies
            fitness_values = []
            for firefly in self.fireflies:
                self._set_weights(firefly)
                fitness = self._calculate_fitness(X, y)
                fitness_values.append(fitness)
                
                if fitness > self.best_fitness:
                    self.best_fitness = fitness
                    self.best_firefly = firefly.copy()
            
            # Move fireflies
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if fitness_values[j] > fitness_values[i]:
                        self.fireflies[i] = self._move_firefly(
                            self.fireflies[i], 
                            self.fireflies[j]
                        )
            
            # Set best weights and evaluate
            self._set_weights(self.best_firefly)
            train_accuracy = self._calculate_fitness(X, y)
            val_accuracy = self._calculate_fitness(X_val, y_val)
            
            history['train_accuracy'].append(train_accuracy)
            history['val_accuracy'].append(val_accuracy)
            
            if (iteration + 1) % 10 == 0:
                print(f"Iteration {iteration + 1}/{self.max_iterations}")
                print(f"Train Accuracy: {train_accuracy:.4f}")
                print(f"Validation Accuracy: {val_accuracy:.4f}")
                
        return history
    
    def predict(self, X):
        self._set_weights(self.best_firefly)
        return self.model.predict(X)

# Example usage
def prepare_data():
    data = pd.read_csv("kc2.csv")
    data['problems'] = data['problems'].apply(lambda x:1 if x=="yes" else 0)
    X = data.drop(['problems'],axis=1)
    y = data['problems']
    y = tf.keras.utils.to_categorical(y)
    
    # scale features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    return train_test_split(X, y, test_size=0.3, random_state=21)

def main():
    #prepare data
    X_train, X_test, y_train, y_test = prepare_data()
#     print(X_train, X_test, y_train, y_test,sep="=="*80)
    model = FireflyOptimizedMLP(
        input_dim=X_train.shape[1],
        hidden_dims=[64, 32],
        output_dim=y_train.shape[1],
        n_fireflies=50,
        max_iterations=50
    )
    
    # Train the model
    history = model.fit(X_train, y_train, X_test, y_test)
    
    # Make predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    
    # Calculate metrics
    test_accuracy = np.mean(predicted_classes == true_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    
    # Confusion Matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    
    # ROC AUC Score (using probabilistic predictions)
    roc_auc = roc_auc_score(y_test, predictions)
    
    # ROC Curve (returned but not plotted in this example)
    fpr, tpr, thresholds = roc_curve(y_test.ravel(), predictions.ravel())
    
    # F-beta Score (with beta=1, which is equivalent to F1 score)
    fbeta = fbeta_score(true_classes, predicted_classes, beta=1, average='weighted')
    
    print(f"\nFinal Test Metrics:")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"ROC AUC Score: {roc_auc:.4f}")
    print(f"F-beta Score (F1): {fbeta:.4f}")
    print(f"\nROC Curve points available ({fpr}, {tpr}, {thresholds})")
    
if __name__ == "__main__":
    main()

Iteration 10/50
Train Accuracy: 0.8274
Validation Accuracy: 0.8535
Iteration 20/50
Train Accuracy: 0.8356
Validation Accuracy: 0.8344
Iteration 30/50
Train Accuracy: 0.8356
Validation Accuracy: 0.8344
Iteration 40/50
Train Accuracy: 0.8384
Validation Accuracy: 0.8535
Iteration 50/50
Train Accuracy: 0.8384
Validation Accuracy: 0.8535
5/5 [==============================] - 0s 4ms/step

Final Test Metrics:
Test Accuracy: 0.8535
Precision: 0.8489
Recall: 0.8535
Confusion Matrix:
[[115  10]
 [ 13  19]]
ROC AUC Score: 0.7734
F-beta Score (F1): 0.8508

ROC Curve points available ([0.         0.14012739 0.14649682 0.14649682 0.15286624 0.15286624
 0.15923567 0.15923567 1.        ], [0.         0.85350318 0.85350318 0.85987261 0.85987261 0.86624204
 0.86624204 0.87898089 1.        ], [2.0000000e+00 1.0000000e+00 9.9715889e-01 2.8411031e-03 2.1598211e-11
 7.1311894e-14 1.2950496e-22 5.2185146e-25 0.0000000e+00])
